# Task 4 — Optimization Model (Linear Programming with PuLP)

**Problem:** A factory produces two products (A and B). Each product uses machine time and labor hours. The factory has limited resources. Goal is to maximize total profit subject to constraints.

This notebook installs PuLP if needed, defines the LP, solves it, and provides functions for sensitivity analysis.

In [ ]:
# Install and import PuLP
try:
    import pulp
except ImportError:
    !pip install pulp
    import pulp

from pulp import LpMaximize, LpProblem, LpVariable, PULP_CBC_CMD, value
import pprint

print('PuLP version:', pulp.__version__)

In [ ]:
# Baseline parameters (editable)
params = {
    'profit_A': 20,       # profit per unit of Product A
    'profit_B': 30,       # profit per unit of Product B
    'machine_A': 2,       # machine hours required per unit A
    'machine_B': 1,       # machine hours required per unit B
    'labor_A': 1,         # labor hours required per unit A
    'labor_B': 2,         # labor hours required per unit B
    'machine_capacity': 100,  # total available machine hours
    'labor_capacity': 80       # total available labor hours
}

pprint.pprint(params)

In [ ]:
# Define and solve the LP
model = LpProblem(name='production-optimization', sense=LpMaximize)

x_A = LpVariable(name='Product_A', lowBound=0, cat='Continuous')
x_B = LpVariable(name='Product_B', lowBound=0, cat='Continuous')

# Objective
model += params['profit_A'] * x_A + params['profit_B'] * x_B, 'Total_Profit'

# Constraints
model += params['machine_A'] * x_A + params['machine_B'] * x_B <= params['machine_capacity'], 'Machine_Time'
model += params['labor_A'] * x_A + params['labor_B'] * x_B <= params['labor_capacity'], 'Labor_Hours'

# Solve
status = model.solve(PULP_CBC_CMD(msg=0))
print('Status:', pulp.LpStatus[status])

print('\nOptimal Production Plan:')
for var in model.variables():
    print(f'  {var.name} = {var.value():.4f}')
print(f'\nTotal Profit = ${value(model.objective):.2f}')

In [ ]:
# Function to solve with custom params and options

def solve_production_problem(params, integer=False, solver_msg=False):
    model = LpProblem(name='production-optimization', sense=LpMaximize)
    cat_type = 'Integer' if integer else 'Continuous'
    x_A = LpVariable(name='Product_A', lowBound=0, cat=cat_type)
    x_B = LpVariable(name='Product_B', lowBound=0, cat=cat_type)
    
    model += params['profit_A'] * x_A + params['profit_B'] * x_B, 'Total_Profit'
    model += params['machine_A'] * x_A + params['machine_B'] * x_B <= params['machine_capacity'], 'Machine_Time'
    model += params['labor_A'] * x_A + params['labor_B'] * x_B <= params['labor_capacity'], 'Labor_Hours'
    
    status = model.solve(PULP_CBC_CMD(msg=1 if solver_msg else 0))
    result = {
        'status': pulp.LpStatus[status],
        'Product_A': x_A.value(),
        'Product_B': x_B.value(),
        'Total_Profit': value(model.objective)
    }
    return result

# Example usage
result_baseline = solve_production_problem(params, integer=False)
print('\nBaseline result:')
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(result_baseline)

In [ ]:
# Sensitivity examples
params_variant = params.copy()
params_variant['labor_capacity'] = 120
result_variant = solve_production_problem(params_variant)
print('\nWhen labor_capacity = 120:')
pp.pprint(result_variant)

params_variant2 = params.copy()
params_variant2['profit_A'] = 40
result_variant2 = solve_production_problem(params_variant2)
print('\nWhen profit_A = 40:')
pp.pprint(result_variant2)

## Notes & How to adapt

- Edit the `params` dictionary to match your real problem (profits, resource usage, capacities).
- To force integer solutions (if units must be whole), call `solve_production_problem(params, integer=True)`.
- To display solver messages, set `solver_msg=True`.
- To add more products or constraints, extend the model-building code accordingly.

If you want, I can also provide a `.py` script version or create a GitHub-ready folder with this notebook and a short README.